In [4]:
import numpy as np

DEBUG = False

## 第四题

In [2]:
def func4(t):
    return np.power(np.e, t) + np.power(np.e, -t)

In [3]:
def method_618(a, b, func):
    """618法"""
    _lambda = a + 0.382 * (b - a)
    _mu = a + 0.618 * (b - a)
    _delta = b - a
    k = 1
    ans = None
    while _delta > 1e-12:
        x1 = func(_lambda)
        x2 = func(_mu)
        if x1 > x2:
            if b - _lambda < 1e-6:
                ans = _mu
                break
            a = _lambda
            _lambda = _mu
            _delta = b - a
            _mu = a + 0.618 * _delta
        else:
            if _mu - a < 1e-6:
                ans = _lambda
                break
            b = _mu
            _mu = _lambda
            _delta = b - a
            _lambda = a + 0.328 * _delta
        k += 1
    
    return ans, func(ans), k

In [4]:
def method_fibonacci(a, b, func):
    """Fibonacci法"""
    DELTA = 1e-6
    fib = [0, 1]
    while fib[-1] < (b - a) / DELTA:
        fib.append(fib[-1] + fib[-2])    
    n = len(fib) - 1
    
    ans = None    
    for k in range(1, n):
        _lambda = a + (b - a) * fib[n - k - 1] / fib[n - k + 1]
        _mu = a + (b - a) * fib[n - k] / fib[n - k + 1]
        if func(_lambda) > func(_mu):
            if b - _lambda < DELTA:
                ans = _mu
                break
            a = _lambda
            _lambda = _mu
            _mu = a + (b - a) * fib[n - k] / fib[n - k + 1]
        else:
            if _mu - a < DELTA:
                ans = _lambda
                break
            b = _mu
            _mu = _lambda
            _lambda = a + (b - a) * fib[n - k - 1] / fib[n - k + 1]

    return ans, func(ans), n

In [5]:
def main4():
    x_star, min_val, k = method_618(-1, 1, func4)
    print(f"618法:\nx_star = {x_star}\nmin_val = {min_val}\nk = {k}\n")
    x_star, min_val, n = method_fibonacci(-1, 1, func4)
    print(f"Fibonacci法:\nx_star = {x_star}\nmin_val = {min_val}\nn = {n}\n")
if not DEBUG:
    main4()

618法:
x_star = 0.0002563395536478077
min_val = 2.0000000657099672
k = 26

Fibonacci法:
x_star = -4.5907169282944493e-07
min_val = 2.0000000000002105
n = 32


## 第五题

In [6]:
def binary_search(a, b, delta=1e-6):
    """二分法"""
    def f(x):
        return x * (x + 2)
    def diff_f(x):
        return 2 * x + 2
    while b - a > delta:
        c = (a + b) / 2
        d = diff_f(c)
        if d == 0:
            return a, b, c
        elif d > 0:
            b = c
        else:
            a = c
    return a, b, (a + b) / 2, f((a + b) / 2)

In [7]:
def main5():
    x, y, z, min_val = binary_search(-3, 6, delta=0.2)
    print(f"二分法:\n区间: [{x}, {y}]\nx_star = {z}\nmin_val = {min_val}\n")   
if not DEBUG:
    main5()

二分法:
区间: [-1.03125, -0.890625]
x_star = -0.9609375
min_val = -0.99847412109375


## 第六题

In [8]:
def func6(t):
    return 1 - t * np.power(np.e, -t * t)

In [9]:
def method_32(a, b, func):
    """三点二次插值法"""
    alpha1, alpha2, alpha3 = a, (a + b) / 2, b
    while True:
        phi1, phi2, phi3 = func(alpha1), func(alpha2), func(alpha3)
        alpha_head = (
            (alpha2 * alpha2 - alpha3 * alpha3) * phi1 +
            (alpha3 * alpha3 - alpha1 * alpha1) * phi2 +
            (alpha1 * alpha1 - alpha2 * alpha2) * phi3
        ) / (
            (alpha2 - alpha3) * phi1 +
            (alpha3 - alpha1) * phi2 +
            (alpha1 - alpha2) * phi3
        ) / 2
        
        phi_head = func(alpha_head)
        
        if np.abs(phi2) < 1e-5 and np.abs(phi2 - phi_head) <= 1e-3:
            return alpha_head, phi_head
        if np.abs(phi2) > 1e-5 and np.abs(phi2 - phi_head) <= 1e-3 * np.abs(phi2):
            return alpha_head, phi_head
        
        if alpha_head > alpha2:
            if phi_head <= phi2:
                alpha1 = alpha2
                alpha2 = alpha_head
            else:
                alpha3 = alpha_head
        else:
            if phi_head <= phi2:
                alpha3 = alpha2
                alpha2 = alpha_head
            else:
                alpha1 = alpha_head

In [10]:
def main6():
    x_star, min_val = method_32(0, 1, func6)
    print(f"三点二次插值法:\nx_star = {x_star}\nmin_val = {min_val}\n")
if not DEBUG:
    main6()

三点二次插值法:
x_star = 0.722782889191233
min_val = 0.5713272614627292


## 第七题

In [10]:
def Goldstein(t0, rou):
    def f(x):
        a = np.array([-2, 21, -60, 50])
        b = np.array([np.power(x, i) for i in range(3, -1, -1)])
        return np.dot(a, b)
    def diff_f(x):
        a = np.array([-6, 42, -60])
        b = np.array([np.power(x, i) for i in range(2, -1, -1)])
        return np.dot(a, b)
    def get_alpha(MIN=0, MAX=1):
        a = MIN
        b = MAX
        alpha = (a + b) / 2

        while True:
            phi_alpha = f(x + alpha * d)
            diff_phi0 = diff_f(x) * d
            phi0 = f(x)
            if phi_alpha <= phi0 + alpha * diff_phi0 * rou:
                if phi_alpha >= phi0 + (1 - rou) * alpha * diff_phi0:
                    return alpha
                else:
                    a = alpha
            else:
                b = alpha
            alpha = (a + b) / 2

    x = t0
    d = 0.5
    while True:
        alpha = get_alpha()
        if alpha < 1e-6:
            break
        x += alpha * d
    return x, f(x)

In [11]:
def Wolfe(t0, rou, sigma):
    def f(x):
        a = np.array([-2, 21, -60, 50])
        b = np.array([np.power(x, i) for i in range(3, -1, -1)])
        return np.dot(a, b)
    def diff_f(x):
        a = np.array([-6, 42, -60])
        b = np.array([np.power(x, i) for i in range(2, -1, -1)])
        return np.dot(a, b)
    def phi_alpha(alpha):
        return f(x + alpha * d)
    def diff_phi_alpha(alpha):
        return diff_f(x + alpha * d) * d
    def get_alpha(MIN=0, MAX=1):
        alpha1 = MIN
        alpha2 = MAX
        phi1 = f(x)
        diff_phi1 = diff_f(x) * d
        alpha = (alpha1 + alpha2) / 2
        
        while True:
            phi = phi_alpha(alpha)
            if phi - phi1 <= rou * alpha * diff_phi1:
                diff_phi = diff_phi_alpha(alpha)
                if diff_phi >= sigma * diff_phi1:
                    return alpha
                else:
                    alpha_head = alpha + (phi - phi1) * diff_phi / (diff_phi1 - diff_phi)
                    alpha1 = alpha
                    phi1 = phi
                    diff_phi1 = diff_phi
                    alpha = alpha_head
            else:
                alpha_head = alpha1 + 0.5 * (alpha - alpha1) / \
                         (1 + (phi1 - phi) / (alpha - alpha1) / diff_phi1)
                alpha2 = alpha
                alpha = alpha_head

    x = t0
    d = 0.25
    while True:
        alpha = get_alpha()
        if alpha < 1e-6:
            break
        x += alpha * d
    return x, f(x)

In [12]:
def main7():
    x_star, min_val = Goldstein(0.5, 0.1)
    print(f"Goldstein 法:\nx_star = {x_star}\nmin_val = {min_val}\n")
    x_star, min_val = Wolfe(0.5, 0.1, 0.8)
    print(f"Wolfe 法:\nx_star = {x_star}\nmin_val = {min_val}\n")
if not DEBUG:
    main7()

Goldstein 法:
x_star = 2.0
min_val = -2.0
Wolfe 法:
x_star = 2.000350282409802
min_val = -1.9999988958060584
